### Modelos de Recuperaciòn de Informaciòn (y evaluaciòn)

Utilizando la colecci´on provista por el equipo docente1
, cuya estructura es la siguiente:
    
vocabulary.txt → [id termino, idf, t´ermino]

documentVectors.txt → [id doc, lista(id terminos)]

queries.txt → [id query, lista(id terminos)]

relevants.txt → [id query, listarelevantes (id doc)]

informationNeeds.txt → [id in, texto libre]


a) Calcule los conjuntos de respuestas usando el modelo booleano y el modelo vectorial (asuma en todos
los casos T F = 1).

In [1]:
from os import listdir
from os.path import isdir
import math


In [ ]:
def indexer(dirname):
    with open(dirname,'r') as dir_indexr:
        

In [2]:
def get_vocabulary (dirname, head = False):
    with open(dirname,'r') as vocabulary:
        words = vocabulary.readlines()
        result=[]
        words= [word.rstrip('\n') for word in words]
        if head:
            words = words[1:]
        
        for word in words:
            items=[]
            word = word.split('\t',)
            for col in word:
                if (len(col)> 0):
                    items.append(col)
            if (len(items) > 0):
                result.append(items)

    return result
    

In [3]:
import numpy as np
from operator import itemgetter

In [4]:
def get_document_vector(dirname, head = False):
    
    trash= ['(',')',',','[',']','\'']
    
    with open(dirname,'r') as documentsfile:
        documents = documentsfile.readlines()
        #result =[]
        documents =[document.rstrip('\n') for document in documents]
        if head:
            documents = documents[1:]
        docs=[]
        
        for document in documents:
            terms = {}
            document = document.split('\t')
            for col in document:
                col = col.replace(' ) ', '')
                col = col.split(':')
                
                value = str(col[1:2])
                for simb in trash:
                    value = value.replace(simb,'')

                #terms[str(col[0:1][0])] = [int(val) for val in value.split()]
                terms["name"] = [str(col[0:1][0])]
                terms["value"] = [int(val) for val in value.split()]
                
                docs.append(terms)
            
        return (docs)
        
            
    

In [5]:
def to_boolean(documents,vocabulary):
    docu = np.array(documents)
    newvocabulary=[]
    #terms =[ (term[2:3] for term in vocabulary)]
    
    for term in vocabulary:   
        newvocabulary.append(term[2:3])

    documents_boolean=[]
    for item in documents:
        #print(item.get('value'))
        boolean_vocabulary = []
        for term in vocabulary:
            if (int(term[0:1][0]) in item.get('value')):
                boolean_vocabulary.append(True)
            else:
                boolean_vocabulary.append(False)
        
        
        documents_boolean.append((item.get('name'),boolean_vocabulary))
    return documents_boolean


In [6]:
voc = (list(get_vocabulary("ejemploRibeiro/vocabulary.txt", True)))
doc = (get_document_vector("ejemploRibeiro/documentVector.txt", True))
doc_bool= to_boolean(doc,voc)

In [7]:
def get_query(dirname,head=False):
    trash= ['(',')',',','[',']','\'']
    
    with open(dirname,'r') as querys_file:
        querys= querys_file.readlines()
            
    querys =[query.rstrip('\n') for query in querys ]
    if head:
        querys = querys[1:]
    query_result=[]

    for query in querys:
        terms = {}
        query = query.split('\t')
        for col in query:
            col = col.replace(' ) ', '')
            col = col.split(':')                

            value = str(col[1:2])
            for simb in trash:
                value = value.replace(simb,'')
        terms["name"] = [str(col[0:1][0])]
        terms["value"] = [int(val) for val in value.split()]

        query_result.append(terms)  
    return query_result
            
            

In [14]:
querys =get_document_vector('ejemploRibeiro/queries.txt',True)

In [15]:
result=[]
for query in querys:
    #result_q=[]
    #print(query)
    query_documents=[]
    for term in query.get('value'):
        for doc_tuple in doc_bool:
            docu, bool_values = doc_tuple
            if bool_values[term-1]:
                if docu[0] not in query_documents:   
                    query_documents.append(docu[0])


    result.append((query.get('name'), query_documents))
print(result)
    


[(['Query 1'], ['Doc 36', 'Doc 17', 'Doc 5', 'Doc 15', 'Doc 25', 'Doc 29', 'Doc 32']), (['Query 2'], ['Doc 1', 'Doc 4', 'Doc 6', 'Doc 9', 'Doc 19', 'Doc 21', 'Doc 29', 'Doc 37', 'Doc 38', 'Doc 14', 'Doc 36']), (['Query 3'], ['Doc 14', 'Doc 19', 'Doc 32', 'Doc 22', 'Doc 8', 'Doc 13', 'Doc 24', 'Doc 27', 'Doc 35', 'Doc 2', 'Doc 11', 'Doc 20', 'Doc 33', 'Doc 34']), (['Query 4'], ['Doc 7', 'Doc 13', 'Doc 27', 'Doc 30', 'Doc 35']), (['Query 5'], ['Doc 10', 'Doc 18', 'Doc 32', 'Doc 8'])]


In [16]:
result=[]
for query in querys:
    query_normal=0
    for term in query.get('value'):
        val=(voc[int(term)-1][1])
        query_normal+=float(val)**2
    query_normal=math.sqrt(query_normal)
    
    result_q=[]
    print(query.get('name'))
    terms_isin_documents={}
    
    for term in query.get('value'):
        query_document=[]
        #query_document=[]
        for doc in doc_bool:
            docu, value = doc
            if (value[term-1]):
                a =[ids for ids, _ in query_document]
                #print (a)
                #print(docu[0])
                if (docu[0] not in [ids for ids, _ in result_q]):
                    query_document.append((docu[0],voc[term-1][1]))
       
            if(term in terms_isin_documents.keys()):
                 #print(terms_isin_documents.keys())
                #print(query_document)
                terms_isin_documents[str(term)].extend(query_document)     
            else:
                if len(query_document)>0:
                    terms_isin_documents[str(term)]=query_document
        result_q.extend(query_document)
    #print("query",query_document) 
    result.append((query.get('name'),result_q))
    
    print("Boolean Result")
    for doc,_ in result_q:
        print(doc)
    #print(result_q)
    query_vector=[]
    for key,value in terms_isin_documents.items():
        query_vector.append(float(voc[int(key)-1][1]))
    
    
    vector_result=[]
    for name,idf in result_q:
        vect_sim =[]
        sim=0
        #value son los documentos que contienen los terminos
        #print(terms_isin_documents.items())
        for key,value in terms_isin_documents.items():
            if (name in [val[0] for val in value]):
                vect_sim.append(float(idf))
            else:
                vect_sim.append(0)
        vector_result.append((name,vect_sim))
    print("------------------------")
    print("Vectorial Result")
    #print(vector_result)
    list_response=[]
    
    for doc,vec in vector_result:
        #r=np.array(r)
        q=0
        
        for idf in vec:
            q+=idf**2
        q = math.sqrt(q)
        q = (np.dot(vec,query_vector)) /(query_normal*q)
        list_response.append((doc,q))
    

    (list_response.sort(key=itemgetter(1),reverse=True))

    for doc in list_response:
        print(doc)
    print("-----------------")
    
    

['Query 1']
Boolean Result
Doc 36
Doc 17
Doc 5
Doc 15
Doc 25
Doc 29
Doc 32
------------------------
Vectorial Result
('Doc 36', 0.6578063799305661)
('Doc 17', 0.6578063799305661)
('Doc 5', 0.36685355803820036)
('Doc 15', 0.36685355803820036)
('Doc 25', 0.36685355803820036)
('Doc 29', 0.36685355803820036)
('Doc 32', 0.36685355803820036)
-----------------
['Query 2']
Boolean Result
Doc 1
Doc 4
Doc 6
Doc 9
Doc 19
Doc 21
Doc 29
Doc 37
Doc 38
Doc 14
Doc 36
------------------------
Vectorial Result
('Doc 14', 0.3651986779831785)
('Doc 36', 0.3651986779831785)
('Doc 1', 0.3485987380748522)
('Doc 4', 0.3485987380748522)
('Doc 6', 0.3485987380748522)
('Doc 9', 0.3485987380748522)
('Doc 19', 0.3485987380748522)
('Doc 21', 0.3485987380748522)
('Doc 29', 0.3485987380748522)
('Doc 37', 0.3485987380748522)
('Doc 38', 0.3485987380748522)
-----------------
['Query 3']
Boolean Result
Doc 14
Doc 19
Doc 32
Doc 22
Doc 8
Doc 13
Doc 24
Doc 27
Doc 35
Doc 2
Doc 11
Doc 20
Doc 33
Doc 34
------------------------

In [115]:
a=[('a',0.4842915343236731),('b',0.7311681192251095),('k',1)]
a.sort(key=itemgetter(1))
print(a)



[('a', 0.4842915343236731), ('b', 0.7311681192251095), ('k', 1)]
